L2-regularized logistic regression for binary or multiclass classification; trains a model (on `train.txt`), optimizes L2 regularization strength on `dev.txt`, and evaluates performance on `test.txt`.  Reports test accuracy with 95% confidence intervals and prints out the strongest coefficients for each class.

In [1]:
from scipy import sparse
from sklearn import linear_model
from collections import Counter
import numpy as np
import operator
import nltk
import math
from scipy.stats import norm

In [2]:
!python -m nltk.downloader punkt

<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/matthewdworkin/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
def load_data(filename):
    X = []
    Y = []
    with open(filename, encoding="utf-8") as file:
        for line in file:
            cols = line.split("\t")
            idd = cols[0]
            label = cols[2].lstrip().rstrip()
            text = cols[3]

            X.append(text)
            Y.append(label)

    return X, Y


In [4]:
class Classifier:

    def __init__(self, feature_method, trainX, trainY, devX, devY, testX, testY):
        self.feature_vocab = {}
        self.feature_method = feature_method
        self.min_feature_count=2
        self.log_reg = None

        self.trainY=trainY
        self.devY=devY
        self.testY=testY
        
        self.trainX = self.process(trainX, training=True)
        self.devX = self.process(devX, training=False)
        self.testX = self.process(testX, training=False)

    # Featurize entire dataset
    def featurize(self, data):
        featurized_data = []
        for text in data:
            feats = self.feature_method(text)
            featurized_data.append(feats)
        return featurized_data

    # Read dataset and returned featurized representation as sparse matrix + label array
    def process(self, X_data, training = False):
        
        data = self.featurize(X_data)

        if training:
            fid = 0
            feature_doc_count = Counter()
            for feats in data:
                for feat in feats:
                    feature_doc_count[feat]+= 1

            for feat in feature_doc_count:
                if feature_doc_count[feat] >= self.min_feature_count:
                    self.feature_vocab[feat] = fid
                    fid += 1

        F = len(self.feature_vocab)
        D = len(data)
        X = sparse.dok_matrix((D, F))
        for idx, feats in enumerate(data):
            for feat in feats:
                if feat in self.feature_vocab:
                    X[idx, self.feature_vocab[feat]] = feats[feat]

        return X


    # Train model and evaluate on held-out data
    def train(self):
        (D,F) = self.trainX.shape
        best_dev_accuracy=0
        best_model=None
        for C in [0.1, 1, 10, 100]:
            self.log_reg = linear_model.LogisticRegression(C = C, max_iter=1000)
            self.log_reg.fit(self.trainX, self.trainY)
            training_accuracy = self.log_reg.score(self.trainX, self.trainY)
            development_accuracy = self.log_reg.score(self.devX, self.devY)
            if development_accuracy > best_dev_accuracy:
                best_dev_accuracy=development_accuracy
                best_model=self.log_reg

#             print("C: %s, Train accuracy: %.3f, Dev accuracy: %.3f" % (C, training_accuracy, development_accuracy))

        self.log_reg=best_model
        

    def test(self):
        return self.log_reg.score(self.testX, self.testY)
        

    def printWeights(self, n=10):

        reverse_vocab=[None]*len(self.log_reg.coef_[0])
        for k in self.feature_vocab:
            reverse_vocab[self.feature_vocab[k]]=k

        # binary
        if len(self.log_reg.classes_) == 2:
              weights=self.log_reg.coef_[0]

              cat=self.log_reg.classes_[1]
              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

              cat=self.log_reg.classes_[0]
              for feature, weight in list(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1)))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

        # multiclass
        else:
          for i, cat in enumerate(self.log_reg.classes_):

              weights=self.log_reg.coef_[i]

              for feature, weight in list(reversed(sorted(zip(reverse_vocab, weights), key = operator.itemgetter(1))))[:n]:
                  print("%s\t%.3f\t%s" % (cat, weight, feature))
              print()

            

In [5]:
def binary_bow_featurize(text):
    feats = {}
    words = nltk.word_tokenize(text)

    for word in words:
        word=word.lower()
        feats[word]=1
            
    return feats



In [6]:
def confidence_intervals(accuracy, n, significance_level):
    critical_value=(1-significance_level)/2
    z_alpha=-1*norm.ppf(critical_value)
    se=math.sqrt((accuracy*(1-accuracy))/n)
    return accuracy-(se*z_alpha), accuracy+(se*z_alpha)

In [7]:
def run(trainingFile, devFile, testFile):
    trainX, trainY=load_data(trainingFile)
    devX, devY=load_data(devFile)
    testX, testY=load_data(testFile)
    
    simple_classifier = Classifier(binary_bow_featurize, trainX, trainY, devX, devY, testX, testY)
    simple_classifier.train()
    accuracy=simple_classifier.test()
    
    lower, upper=confidence_intervals(accuracy, len(testY), .95)
    print("Test accuracy for best dev model: %.3f, 95%% CIs: [%.3f %.3f]\n" % (accuracy, lower, upper))

    simple_classifier.printWeights()
    


# Make train test splits

In [22]:
import pandas as pd
adj_data = pd.read_table("./AP2/adjudicated_data.txt")
adj_data

,1,adjudicated,humor,woman who teaches special-needs children killing it at dinner party
0,2,adjudicated,satire,dnc criticized for overly restrictive debate r...
1,3,adjudicated,irony,custodian taken into custody
2,4,adjudicated,none,mike pence takes oath of office as country's n...
3,5,adjudicated,humor,bedtime story from fucking bible again
4,6,adjudicated,satire,literary theorists admit they still have no id...
...,...,...,...,...
494,496,adjudicated,none,longing to finally visit the cuba of my dreams
495,497,adjudicated,none,trump's evangelical advisors urged him to prot...
496,498,adjudicated,none,flaws in how we evaluate leaders (from kahnema...
497,499,adjudicated,irony,intel unveils oversized novelty processor


In [15]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(adj_data, test_size=100)
train, dev = train_test_split(train, test_size=100)
len(train), len(dev), len(test)

(299, 100, 100)

In [25]:
train.to_csv("./splits/train.txt", sep="\t", index=False)
dev.to_csv("./splits/dev.txt", sep="\t", index=False)
test.to_csv("./splits/test.txt", sep="\t", index=False)

In [26]:
trainingFile = "splits/train.txt"
devFile = "splits/dev.txt"
testFile = "splits/test.txt"
    
run(trainingFile, devFile, testFile)


Test accuracy for best dev model: 0.475, 95% CIs: [0.378 0.573]

humor	0.900	woman
humor	0.830	she
humor	0.828	study
humor	0.782	man
humor	0.750	teacher
humor	0.719	god
humor	0.707	dressing
humor	0.697	like
humor	0.689	road
humor	0.668	fucking

irony	1.138	hero
irony	0.926	sleep
irony	0.791	netflix
irony	0.765	wishes
irony	0.712	up
irony	0.697	floor
irony	0.610	introduces
irony	0.565	how
irony	0.562	have
irony	0.534	place

none	0.954	we
none	0.829	a
none	0.766	.
none	0.748	mississippi
none	0.724	twitter
none	0.670	gop
none	0.624	florida
none	0.617	us
none	0.590	homeless
none	0.584	west

satire	0.904	now
satire	0.903	really
satire	0.754	about
satire	0.751	u.s.
satire	0.716	features
satire	0.706	doctors
satire	0.697	conscious
satire	0.695	$
satire	0.673	sentenced
satire	0.672	school



## Improving on the baseline